In [1]:
import os     ## Allows operating system commands to be performed though Python
import sys    ## Allows you to exit a Python script for testing
import shutil ## Allows movement of files from one directory to another
import  glob  ## Global searching of strings as files

In [3]:
#change directory to folder with downloaded pdf's
#os.chdir('C:\\Users\\akomandu\\PDF_Scrapes\\Hewlett_Packard_MSDS')
os.chdir('C:\\Users\\akomandu\\Python Portfolio\\PDF_Scrapes\\CDC_HHE')

In [4]:
pwd=os.getcwd()

In [5]:
#set exec path to folder with pdf-to-text application
#execpath = 'C:\\Users\\akomandu\\PDF_Scrapes\\xpdfbin-win-3.04\\bin64\\'
execpath = 'C:\\Users\\akomandu\\Python Portfolio\\PDF_Scrapes\\xpdfbin-win-3.04\\bin64\\'

In [6]:
#set names of applications 
htmlexec = "pdftohtml.exe"
txtexec = 'pdftotext.exe'

In [7]:
#set files to convert
#files = glob.glob('C:\\Users\\akomandu\\PDF_Scrapes\\Hewlett_Packard_MSDS\\*.pdf')
files = glob.glob('C:\\Users\\akomandu\\Python Portfolio\\PDF_Scrapes\\Hewlett_Packard_MSDS\\*.pdf')

In [9]:
#convert pdf files to text files
for f in files[0:2]:
#    filename = os.path.join("C:\\Users\\akomandu\\PDF_Scrapes\\Hewlett_Packard_MSDS", os.path.basename(f))
    filename = os.path.join("C:\\Users\\akomandu\\", os.path.basename(f))
    dirname = os.path.basename(f)[:-4]
    
    txtcmd = os.path.join(execpath,txtexec)

    txtcmd = " ".join([txtcmd,"-table","-nopgbrk",f])

    os.system(txtcmd)


## Data Extraction

In [48]:
import os     ## Allows operating system commands to be performed though Python
import sys    ## Allows you to exit a Python script for testing
import shutil ## Allows movement of files from one directory to another
import string ## Fancy string manipulations
import  glob  ## Global searching of strings as files
import pandas as pd
import string

In [49]:
#create function for cleaning lines
clean = lambda dirty: ''.join(filter(string.printable.__contains__, dirty))

In [50]:
#set path to text files
pwd = ['C:\\Users\\akomandu\\PDF_Scrapes\\Hewlett_Packard_text']
os.chdir("".join(pwd))
pwd = os.getcwd()

In [51]:
#find all text files
files = glob.glob('*.txt')

In [ ]:
#create lists for pertinent information in each text file
Product_Name = []
Chemical_Name = []
Manufacturer = []
CAS_Number = []
Percentage = []
Files = []

#iterate through each text file to clean lines and append relevant information to lists
for file in files:
    print(file)
    
    ifile = open(file)    
    for line in ifile:
        if line == '/n': continue
        cline = clean(line)
        cline = cline.lower()
        cline = cline.replace(',','_')
        cline = cline.replace(';','_')
        cline = cline.strip()

        cline = cline.split("  ")
        cline = [x.strip() for x in cline if x != ""]
#        print(cline)

    #hp_2
        if 'product name' in str(cline):
            name=' / '.join(cline[1:])
        if 'trade name' in str(cline):
            name=' / '.join(cline[1:])
        if 'product identifier' in str(cline):
            name=' / '.join(cline[1:])

        if 'cas number' in cline:
            list1 = []
            for line in ifile:
                dline = clean(line)
                dline = dline.lower()
                dline = dline.replace(',','_')
                dline = dline.replace(';','_')
                dline = dline.strip()
                dline  = dline.split("  ")
                dline = [x.strip() for x in dline if x != ""]
                list1.append(dline)

                if 'subject' in str(line).lower():
                    break
                elif len(dline) == 1:
                    break
            list1 = list1[:-1]
            list2 = [x for x in list1 if x != []]
            
            for j in list2:
                if len(j) == 3:
                    try: 
                        Product_Name.append(name)
                    except:
                        Product_Name.append(' ')
                    try:
                        Chemical_Name.append(j[0])
                    except IndexError:
                        Chemical_Name.append(' ')
                    try:
                        CAS_Number.append(j[1])
                    except IndexError:
                        CAS_Number.append(' ')
                    try :
                        Percentage.append(j[2])
                    except IndexError:
                        Percentage.append(' ')
                    Files.append(file)

                elif len(j) == 4:
                    try: 
                        Product_Name.append(name)
                    except:
                        Product_Name.append(' ')
                    try:
                        Chemical_Name.append(j[0])
                    except IndexError:                                
                        Chemical_Name.append(' ')
                    try:
                        CAS_Number.append(j[2])
                    except IndexError:
                        CAS_Number.append(' ')
                    try :
                        Percentage.append(j[3])
                    except IndexError:
                        Percentage.append(' ')
                    Files.append(file)
    name = 'None'

In [ ]:
#create dataframe to display scraped data in the lists
Test_Run = pd.DataFrame({'Product_Name': Product_Name, 'Chemical_Name': Chemical_Name, 'CAS_Number': CAS_Number, 'Percentage': Percentage, "File": Files})
Test_Run = Test_Run.drop_duplicates(keep = 'first')
Test_Run

## Filter through dataframe to remove extraneous information (e.g. remove words and sentences that were accidentally included in the dataframe)

In [68]:
Test2 = Test_Run[Test_Run['CAS_Number'].map(lambda x: 'issue date' not in x.split(':'))]

In [ ]:
Test2['CAS_Number'].value_counts()

In [70]:
Test3 = Test2[Test2['CAS_Number'].map(lambda x: x != 'this product has been evaluated using criteria specified in 29 cfr 1910.1200')]

In [ ]:
Test3[Test3['CAS_Number'].map(lambda x: '<' in x)]

In [ ]:
Test2[Test2['File'] == 'hp_2904.txt']

In [84]:
Test4 = Test3[Test3['Product_Name'] != 'None']

In [89]:
stopwords = ['%', 'concentration range', 'cas number']

In [102]:
Test5 = Test4[Test4['CAS_Number'].map(lambda x: x not in stopwords)]

In [103]:
Test6 = Test5[Test5['CAS_Number'].map(lambda x: '%' not in x)]

In [105]:
Test7 = Test6[Test6['CAS_Number'].map(lambda x: x != 'similar chemical')]

In [115]:
#create function to export dataframe to excel file
def checkfile(file_name):
    """
    Usage:
    ------
    checkfile(file_name)

    Description:
    ------------
    Function to check if a file name exists in the current working
    directory and rename the new file, rather than overwrite the old file
    with new data.

    Parameters:
    -----------
    file_name : string, database name tag in .my.cnf file (default=None)
    out       : string, a unique file name based on the provided file_name;
                uniqueness is insured by incrementing a number appended to the
                file by 1
    """
    cwd = os.getcwd()
    cwf = os.path.join(cwd,file_name)
    if not os.path.exists(cwf):
        return cwf
    root, ext = os.path.splitext(os.path.expanduser(cwf))
    dir = os.path.dirname(root)
    fname = os.path.basename(root)
    candidate = fname+ext
    index = 1
    ls = set(os.listdir(dir))
    while candidate in ls:
        candidate = "{}_{}{}".format(fname,index,ext)
        index += 1
    return os.path.join(dir,candidate)

def to_excel(df,xlfile,sheet_name='Sheet 1',cas_col='casrn',index=False,filecheck=False,encoding=None):
    """
    Usage:
    ------
    to_excel(df,xlfile)

    Description:
    ------------
    Writes Pandas dataframe to MS Excel file. This differs from Pandas' own
    to_excel function in that it will find the 'casrn' column and write it to
    the file as a string instead of a number, thus preventing some CASRNs to be
    autoformatted to strings.

    Parameters:
    -----------
    df              pandas.DataFrame, object to write to an MS Excel file
    xlfile          string, name of MS Excel file to generate
    sheet_name      string, name of MS Excel sheet the dataframe will be written to
    cas_col         string, name of the column in the dataframe containing CASRNs (default:'casrn')
    index           boolean, default True write row names (index)
    checkfile       check if a file by the same name already exists in the path

    """
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    if cas_col not in df.columns.tolist():
        print("Error! 'casrn' column not in dataframe. Use Pandas' to_excel function.")
        return

    cas_col = alphabet[df.columns.get_loc(cas_col)].upper()
    if filecheck:
        writer = pd.ExcelWriter(checkfile(xlfile),engine="xlsxwriter")
    else:
        writer = pd.ExcelWriter(xlfile,engine="xlsxwriter")
    df.to_excel(writer,sheet_name=sheet_name,index=index,encoding=encoding)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    format1 = workbook.add_format({"bold":False,"font_color":"black"})
    worksheet.set_column(cas_col+':'+cas_col,12,format1)
    writer.save()
    return

In [116]:
to_excel(Test7, 'HP_Scraped_Data_Sans_Batteries.xlsx', cas_col = "CAS_Number")